In [2]:
%matplotlib inline

In [3]:
import  os
import  sys
import  glob

In [63]:
import  pdfkit
import  fitsio
import  redrock
import  datetime
import  redrock.templates

import  pandas                  as      pd
import  pylab                   as      pl
import  numpy                   as      np
import  matplotlib.pyplot       as      plt
import  astropy.io.fits         as      fits
import  matplotlib.pyplot       as      plt
import  astropy.io.fits         as      fits

from    astropy.table           import  Table, join, vstack, unique
from    astropy.coordinates     import  SkyCoord
from    astropy                 import  units as u
from    desitarget.cmx          import  cmx_targetmask
from    desitarget.targets      import  desi_mask, bgs_mask, mws_mask
from    desispec.interpolation  import  resample_flux
from    desispec.resolution     import  Resolution
from    desispec.specscore      import  compute_frame_scores
from    collections             import  OrderedDict
from    mpl_toolkits.axes_grid1 import  make_axes_locatable
from    itertools               import  combinations
from    desisurvey.utils        import  get_date
from    desitarget.sv1          import  sv1_cuts 

In [5]:
##  DR7 cutouts
import  requests

from    PIL                     import  Image
from    io                      import  BytesIO

In [7]:
# Prospect
sys.path.append("/global/homes/m/mjwilson/BGS/MINISV/prospect/py/")

import desispec
import prospect

from   prospect                import utils_specviewer, plotframes

In [8]:
plt.rcParams['figure.figsize'] = [10, 10]

In [9]:
def flux2mag(flux) :
    mag = -2.5*np.log10(flux*(flux>0) + 0.001*(flux<=0)) + 22.5
    mag[(flux<=0)] = 0.
    
    return  mag

##  DR7 cutouts 

In [10]:
def get_dr7cutout(ra, dec):
  nim         = len(ra)
  nrow        =  np.maximum(np.floor(nim / 5).astype(np.int), 1)

  fig, axes   = plt.subplots(nrow, 5, figsize=(20, 5 * nrow))

  row         = 0
  col         = 0

  nmax        = np.minimum(5 * nrow, nim)
  
  for i in range(nmax):        
    url       = 'http://legacysurvey.org/viewer/jpeg-cutout?ra={:.4f}&dec={:.4f}&layer=decals-dr7&pixscale=0.27&bands=grz'.format(ra[i], dec[i])
    response  = requests.get(url)
    img       = Image.open(BytesIO(response.content))

    axes[row, col].imshow(img)
    
    col      += 1
    
    if col > 4:
        row  += 1
        col   = 0 

##  Night log.
http://desi-www.kpno.noao.edu:8090/nightsum/nightsum-2020-02-25/nightsum.html

##  Mini SV page.
https://desi.lbl.gov/trac/wiki/TargetSelectionWG/miniSV2

#  Summary 

In [11]:
# /global/homes/m/mjwilson/BGS/MINISV/zhou
version = 'v2'
root    = '/global/cscratch1/sd/mjwilson/BGS/MINISV/coadds/{}/'.format(version)

## Prep. redrock templates

In [12]:
templates = dict()

for filename in redrock.templates.find_templates():
    t = redrock.templates.Template(filename)
    templates[(t.template_type, t.sub_type)] = t

DEBUG: Read templates from /global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/code/redrock-templates/master
DEBUG: Using default redshift range -0.0050-1.6997 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.0500-5.9934 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-CV.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-M.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-WD.fits


In [23]:
cframes = Table(fits.open('/global/homes/m/mjwilson/BGS/MINISV/zhou/bgs_allcframes_{}_gfa.fits'.format(version))[1].data) 
cframes

NIGHT,MJD,LAT,LON,ELV,TILEID,EXPID,EXPTIME,CAMERA,PROGRAM,PETAL_LOC,SPECGRPH,RA,DEC,MOON_ALT,MOON_RA,MOON_DEC,MOON_FRAC,MOON_SEP,SUN_ALT,SUN_RA,SUN_DEC,SUN_SEP,AIRMASS,EBV,MEDIAN,MEAN,N_NON_FINITE,MAX,MIN,SIG_ROBUST,SIG,MEDIAN_Q1,MEAN_Q1,N_NON_FINITE_Q1,MAX_Q1,MIN_Q1,SIG_ROBUST_Q1,SIG_Q1,MEDIAN_Q2,MEAN_Q2,N_NON_FINITE_Q2,MAX_Q2,MIN_Q2,SIG_ROBUST_Q2,SIG_Q2,MEDIAN_Q3,MEAN_Q3,N_NON_FINITE_Q3,MAX_Q3,MIN_Q3,SIG_ROBUST_Q3,SIG_Q3,MEDIAN_Q4,MEAN_Q4,N_NON_FINITE_Q4,MAX_Q4,MIN_Q4,SIG_ROBUST_Q4,SIG_Q4,SKY_MAG_AB,SKY_MAG_AB_PER_AMP,SKYRA,SKYDEC,MOONRA,MOONDEC,T_C_FOR_DARK,T_C_FOR_DARK_IS_GUESS,TIME_S_FOR_DARK,CUBE_INDEX,RACEN,DECCEN,CONTRAST,FIBER_FRACFLUX,N_SOURCES_FOR_PSF,APER_CORR_FAC,XCENTROID_PSF,YCENTROID_PSF,MOON_SEP_DEG,ZP_ADU_PER_S,TRANSPARENCY,NPIX_BAD_TOTAL,NPIX_BAD_PER_AMP,OVERSCAN_MEDIANS_ADU,PRESCAN_MEDIANS_ADU,FWHM_MAJOR_PIX,FWHM_MINOR_PIX,FWHM_PIX,FWHM_ASEC,N_SOURCES,NAXIS,CD,CDELT,CRPIX,CRVAL,LONGPOLE,LATPOLE,PV2,DO_FIT_DARK_SCALING,MASTER_DARK_EXPTIME,MASTER_DARK_GCCDTEMP,DARK_TEMP_SCALING_FACTOR,TOTAL_DARK_SCALING_FACTOR,DARK_RESCALE_FACTORS_PER_AMP,DARK_RESCALE_FACTOR_BESTFIT,DARK_RESCALE_FACTOR_ADOPTED,APPLY_DARK_RESCALE_FACTOR,DARK_RESCALE_NCALLS,DARK_RESCALE_CONVERGED
str40,float64,float64,float64,float64,int64,int64,float64,str40,str40,int32,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20200225,58905.25690431,31.96403,-111.59989,2097.0,70502,52090,900.0,b,SV tile 70502,0,0,180.012,-0.510261,-34.718022305904064,6.5807533473741975,-2.8514428140522536,0.0627905403740725,172.62306614298288,-58.985404389203396,338.8184557889157,-8.90282836156349,156.88472549965033,1.6491293775976599,0.031056919357460633,1498.5,1498.71237064332,0.0,14171.5,1305.5,74.1301109252801,74.1301109252801,1510.5,1512.1791519046753,0.0,6803.5,1436.0,10.378215529539215,10.378215529539215,1448.0,1453.5707260250122,0.0,13751.0,1386.0,10.378215529539215,10.378215529539215,1506.5,1508.7537964813469,0.0,7934.5,1429.0,12.602118857297617,12.602118857297617,1377.0,1381.3672603636749,0.0,7951.5,1305.5,11.119516638792016,11.119516638792016,21.166722602992905,21.162157,180.012,-0.510261,6.887381,-2.523766,9.646,0.0,5.0,62.0,179.99966051964955,-0.5499658695163923,3.8426876448623024,0.15819694491469147,4.0,0.4802858062435611,24.880265561743073,24.893638934214035,172.3655579744284,25.4973871698881,0.9372713127978513,0.0,0.0,1407.0,1394.0,12.65485591453827,11.17607840033076,11.847539901733398,2.4287456798553464,11.0,1540.0,2.0343276314399996e-08,1.0,16266.06196518,89.7508695,180.0,90.0,0.0,1.0,5.0,10.07,0.887832033298626,4.692542112685003,1.0497448730468755,1.050003051757813,1.050003051757813,1.0,36.0,1.0
20200225,58905.29573492,31.96403,-111.59989,2097.0,70502,52112,900.0,b,SV tile 70502,0,0,180.006929,-0.50838,-44.78323495364497,7.13291666046073,-2.6659189741928837,0.06456711009394417,172.20097121173714,-65.64763237733636,338.8556693565404,-8.888320131591259,156.92963241839925,1.369023183589419,0.0312053262371375,1499.0,1499.251294276511,0.0,15046.0,1307.0,73.3888098160273,73.3888098160273,1511.0,1512.3986986736918,0.0,6854.5,1438.0,10.378215529539215,10.378215529539215,1448.5,1453.66285125969,0.0,14277.5,1389.0,11.860817748044816,11.860817748044816,1508.0,1510.5980640973232,0.0,8373.5,1431.0,12.602118857297617,12.602118857297617,1378.0,1

In [24]:
# BGS EXPIDS:  defined by night and tile.
bgs_expids = cframes['EXPID']
nights     = cframes['NIGHT'] 
tiles      = cframes['TILEID']
petals     = cframes['PETAL_LOC']

##  Unique tiles and nights.

In [25]:
samples    = OrderedDict(set(zip(nights, tiles)))
order      = np.sort([np.int(x) for x in samples.keys()]).astype(str)

samples    = OrderedDict(zip(order, [samples[x] for x in order]))
samples    = list(zip(samples.keys(), samples.values()))
samples

[('20200225', 70502),
 ('20200227', 70502),
 ('20200228', 70500),
 ('20200303', 70500)]

In [26]:
# MOONRA, MOONDEC, MOON_SEP_DEG
attributes = ['MOON_ALT', 'MOON_RA', 'MOON_DEC', 'MOON_FRAC', 'MOON_SEP', 'SUN_ALT',\
              'SUN_RA', 'SUN_DEC', 'SUN_SEP', 'EXPTIME', 'AIRMASS']

# 'SKY_MAG_AB', 'FIBER_FRACFLUX', 'MOON_SEP_DEG', 'TRANSPARENCY', 'FWHM_ASEC']

##  GET COORDINATES FILE FOR FIBER REACH ERRORS
https://desi.lbl.gov/trac/wiki/FPS/PositionerFlags

##   GET COADDED EXPOSURES AND ZBEST FILES

In [27]:
utiles                     = np.unique(tiles)
upetals                    = np.array([0,3,6,7,9])
unights                    = np.unique(nights)

In [28]:
# https://github.com/desihub/desispec/blob/c544749a8ab4b982a054b91247f056c2ce4bd8ce/py/desispec/specscore.py#L148 
bandpasses                 = {"b": [4000, 5800], "r": [5800, 7600], "z": [7600, 9800]}

fpath                      = root + '/NEXP1/{}/{}/coadd-{}-0-{:08d}.fits'.format(70500, '20200303', '20200303', 52933)
wave                       = fitsio.read(fpath, ext=2)

results                    = OrderedDict()

for tile in utiles:
    results[tile]          = OrderedDict()

for i, (night, tile) in enumerate(samples):
      results[tile][night] = OrderedDict()

      for ncoadd in [1, 2, 4, 'ALL']:
        ##  E.g.  coadd-20200303-6-2expc.fits
        ##        coadd-20200303-3-00052933.fits 
        ##
        ##        coadd-20200303-6-3exp-subset-2.fits
        if ncoadd in [1, 2, 4]:
          look               =  glob.glob(root + '/NEXP{}/{}/{}/coadd-*'.format(ncoadd, tile, night)) 
          subsets            = [x.split('-')[-1].replace('.fits', '') for x in look]
          subsets            =  np.array([int(x) for x in subsets])                
   
          try:
            maxn_splits      = subsets.max()

          except:
            print('{}    No coadd available for tile {} night {} and ncoadd {} ({})'.format(str(datetime.datetime.now()), tile, night, ncoadd, subsets))

            key                                    = 'NEXP' + str(ncoadd)
            
            results[tile][night][key]              = None
          
            continue
     
          if ncoadd == 1:
            keys                                   = ['NEXP' + str(ncoadd) + '-' + str(x) for x in subsets]

          else:
            keys                                   = ['NEXP' + str(ncoadd) + '-' + str(nsplit) for nsplit in range(maxn_splits)]      

          nsplits                                  =  list(range(maxn_splits))
    
        elif ncoadd == 'ALL':
          keys                                     = ['ALL']
          nsplits                                  = [-99]

        else:
          raise  ValueError('Problem.')
                 
        for key, nsplit in zip(keys, nsplits): 
          results[tile][night][key]                 = {}
        
          results[tile][night][key]['coadds']       = []
          results[tile][night][key]['coadds_flux']  = []
          results[tile][night][key]['coadds_ivar']  = []
          results[tile][night][key]['coadds_mask']  = []
          results[tile][night][key]['coadds_res']   = []
          results[tile][night][key]['fiberstatus']  = []
          results[tile][night][key]['zbests']       = []
                
          results[tile][night][key]['scores']       = []
            
          results[tile][night][key]['coadds_paths'] = {}
          results[tile][night][key]['zbests_paths'] = {}
          
          for petal in upetals:
            print('{}    Solving for tile {}, night {}, key {}, petal {} and ncoadd {}.'.format(str(datetime.datetime.now()), tile, night, key, petal, ncoadd))
            
            if key == 'ALL':
              ##  FIRST EXT: FIBER MAPS
              coadd_path = root + '/ALL/{}/{}/coadd-{}-{}-allexp.fits'.format(tile, night, night, petal) 
              zbest_path = root + '/ALL/{}/{}/coadd-{}-{}-allexp.fits'.format(tile, night, night, petal).replace('coadd', 'zbest')
            
            elif key.split('-')[0] == 'NEXP1':
              coadd_path = root + '/NEXP{}/{}/{}/coadd-{}-{}-{:08d}.fits'.format(ncoadd, tile, night, night, petal, np.int(key.split('-')[-1]))
              zbest_path = root + '/NEXP{}/{}/{}/coadd-{}-{}-{:08d}.fits'.format(ncoadd, tile, night, night, petal, np.int(key.split('-')[-1])).replace('coadd', 'zbest')
            
            else:
              ##  FIRST EXT: FIBER MAPS
              coadd_path = root + '/NEXP{}/{}/{}/coadd-{}-{}-{}exp-subset-{}.fits'.format(ncoadd, tile, night, night, petal, ncoadd, nsplit) 
              zbest_path = root + '/NEXP{}/{}/{}/coadd-{}-{}-{}exp-subset-{}.fits'.format(ncoadd, tile, night, night, petal, ncoadd, nsplit).replace('coadd', 'zbest')
            
            results[tile][night][key]['coadds_paths'][petal] = coadd_path
            results[tile][night][key]['zbests_paths'][petal] = zbest_path
            
            try:
              coadd = fitsio.read(coadd_path)
              zbest = fitsio.read(zbest_path)
    
            except:
              results[tile][night][key]['coadds_paths'][petal] = None
              results[tile][night][key]['zbests_paths'][petal] = None
            
              continue
    
            results[tile][night][key]['coadds'].append(coadd)  
            results[tile][night][key]['zbests'].append(zbest)
    
            _ = fitsio.read(coadd_path, ext=2) 
         
            assert  np.all(np.isclose(wave, _))
        
            results[tile][night][key]['coadds_flux'].append(fitsio.read(coadd_path, ext=3)) 
            results[tile][night][key]['coadds_ivar'].append(fitsio.read(coadd_path, ext=4))
            results[tile][night][key]['coadds_mask'].append(fitsio.read(coadd_path, ext=5))
            results[tile][night][key]['coadds_res'].append(fitsio.read(coadd_path,  ext=6))
            results[tile][night][key]['fiberstatus'].append(fitsio.read(coadd_path, ext=1)['FIBERSTATUS'])
        
            results[tile][night][key]['scores'].append(fitsio.read(coadd_path,  ext=7))

          ##
          results[tile][night][key]['coadds']      = np.hstack(results[tile][night][key]['coadds'])
          results[tile][night][key]['zbests']      = np.hstack(results[tile][night][key]['zbests'])
          results[tile][night][key]['coadds_flux'] = np.vstack(results[tile][night][key]['coadds_flux'])
          results[tile][night][key]['coadds_ivar'] = np.vstack(results[tile][night][key]['coadds_ivar'])
          results[tile][night][key]['coadds_mask'] = np.vstack(results[tile][night][key]['coadds_mask']) 
          results[tile][night][key]['coadds_res']  = np.vstack(results[tile][night][key]['coadds_res'])
          results[tile][night][key]['fiberstatus'] = np.hstack(results[tile][night][key]['fiberstatus'])
        
          results[tile][night][key]['scores']      = np.vstack(results[tile][night][key]['scores'])
  
          dat                                      = fits.open(coadd_path)

          ids_incoadd                              = np.unique([np.int(dat[0].header['INFIL*'][x].split('-')[-1].replace('.fits', '')) for x in range(len(dat[0].header['INFIL*']))])
            
          keep                                     = [x in ids_incoadd for x in cframes['EXPID']]   
          _                                        =  cframes[keep]
    
          for attribute in attributes: 
              results[tile][night][key][attribute.upper()] = np.array(_[attribute]).astype(np.float).mean()

          if key == 'ALL':            
            results[tile][night]['ALL-NEXP{}'.format(len(ids_incoadd))] = results[tile][night].pop(key)
                
          ##  for key in ['coadds', 'zbests', 'coadds_flux', 'coadds_ivar', 'coadds_mask', 'coadds_res']:
          ##    print(results[tile][night][key].shape)
        
print('\n\nDone.\n\n')

2020-03-11 18:08:14.975441    Solving for tile 70502, night 20200225, key NEXP1-52115, petal 0 and ncoadd 1.
2020-03-11 18:08:15.350405    Solving for tile 70502, night 20200225, key NEXP1-52115, petal 3 and ncoadd 1.
2020-03-11 18:08:15.719209    Solving for tile 70502, night 20200225, key NEXP1-52115, petal 6 and ncoadd 1.
2020-03-11 18:08:16.074723    Solving for tile 70502, night 20200225, key NEXP1-52115, petal 7 and ncoadd 1.
2020-03-11 18:08:16.843150    Solving for tile 70502, night 20200225, key NEXP1-52115, petal 9 and ncoadd 1.
2020-03-11 18:08:18.262596    Solving for tile 70502, night 20200225, key NEXP1-52114, petal 0 and ncoadd 1.
2020-03-11 18:08:19.129596    Solving for tile 70502, night 20200225, key NEXP1-52114, petal 3 and ncoadd 1.
2020-03-11 18:08:20.012901    Solving for tile 70502, night 20200225, key NEXP1-52114, petal 6 and ncoadd 1.
2020-03-11 18:08:21.010592    Solving for tile 70502, night 20200225, key NEXP1-52114, petal 7 and ncoadd 1.
2020-03-11 18:08:22

In [29]:
for i, (night, tile) in enumerate(samples):
  keys = results[tile][night].keys()

  print('------------- {} -------------- {} ------------------'.format(night, tile))
  print(keys)
  print('\n')

------------- 20200225 -------------- 70502 ------------------
odict_keys(['NEXP1-52115', 'NEXP1-52114', 'NEXP1-52113', 'NEXP1-52116', 'NEXP1-52112', 'NEXP1-52090', 'NEXP2-0', 'NEXP2-1', 'ALL-NEXP6'])


------------- 20200227 -------------- 70502 ------------------
odict_keys(['NEXP1-52447', 'NEXP1-52448', 'NEXP4', 'ALL-NEXP2'])


------------- 20200228 -------------- 70500 ------------------
odict_keys(['NEXP1-52636', 'NEXP1-52630', 'NEXP4', 'ALL-NEXP2'])


------------- 20200303 -------------- 70500 ------------------
odict_keys(['NEXP1-52932', 'NEXP1-52933', 'NEXP1-52939', 'NEXP1-52942', 'NEXP1-52937', 'NEXP1-52943', 'NEXP1-52935', 'NEXP1-52940', 'NEXP1-52938', 'NEXP1-52941', 'NEXP1-52934', 'NEXP1-52936', 'NEXP2-0', 'NEXP2-1', 'NEXP2-2', 'NEXP2-3', 'NEXP2-4', 'NEXP4-0', 'NEXP4-1', 'ALL-NEXP12'])




##  N BGS TARGETS, WITH NO FIBER ERROR, AND GOOD Z, OF TYPE MAIN BRIGHT, OF TYPE SV0

In [72]:
# 'MOON_SEP_DEG', 'TRANSPARENCY', 'SKY_MAG_AB', 'FIBER_FRACFLUX', 'FWHM_ASEC'.
print_cols                = ['EXPTIME', 'MOON_FRAC', 'MOON_ALT', 'MOON_SEP', 'AIRMASS', 'SUN_ALT', 'SUN_SEP']

toprint                   = {}

toprint['NIGHT']          = []
toprint['TILE']           = []
toprint['NEXP']           = []

for x in print_cols:
  toprint[x]              = []

toprint['IS_GOOD']        = []
toprint['IS_BGS']         = []
toprint['IS_GOOD_BGS']    = []
toprint['IS_ZGOOD_BGS']   = []
toprint['IS_ZGOOD_BGS_%'] = []
toprint['IS_GOOD_MAIN']   = []
toprint['IS_ZGOOD_MAIN']  = []
toprint['IS_GOOD_FAINT']  = []
toprint['IS_ZGOOD_FAINT'] = []
toprint['IS_GOOD_LOWQ']   = []
toprint['IS_ZGOOD_LOWQ']  = []
toprint['IS_GOOD_FIB']    = []
toprint['IS_ZGOOD_FIB']   = []

for i, (night, tile) in enumerate(samples):
  keys = results[tile][night].keys()
    
  for key in keys:
    if results[tile][night][key] == None:
        continue
        
    isgood                    = (results[tile][night][key]['fiberstatus'] == 0)

    coadds                    =  results[tile][night][key]['coadds']
    zbest                     =  results[tile][night][key]['zbests']
     
    IS_SV0                    = (coadds['CMX_TARGET'] & cmx_targetmask.cmx_mask.mask('SV0_BGS')) != 0

    IS_GOOD_BGS               = IS_SV0 & isgood
    IS_GOOD_MAIN              = IS_GOOD_BGS & ((coadds['CMX_TARGET'] & cmx_targetmask.cmx_mask.mask('MINI_SV_BGS_BRIGHT')) != 0)
    
    # Exclude QSOs (lack of redshift precision for small scales).
    IS_ZGOOD_BGS              = IS_GOOD_BGS  & (zbest['ZWARN'] == 0) & (zbest['SPECTYPE'] == b'GALAXY') 
    IS_ZGOOD_BGS              = IS_ZGOOD_BGS & (zbest['ZERR'] < 0.0005 * (1. + zbest['Z']))
    IS_ZGOOD_BGS              = IS_ZGOOD_BGS & (zbest['Z'] < 0.5)
    
    IS_ZGOOD_MAIN             = IS_ZGOOD_BGS & ((coadds['CMX_TARGET'] & cmx_targetmask.cmx_mask.mask('MINI_SV_BGS_BRIGHT')) != 0)
    
    ##
    sv_types                  = ['bright', 'faint', 'faint_ext', 'lowq', 'fibmag']
    bgs_subsample             = {}
    
    for targtype in sv_types:      
      # No extinction. 
      rmag = flux2mag(coadds['FLUX_R'])
      grr  = coadds['GAIA_PHOT_G_MEAN_MAG'] - rmag                                                                                
        
      bgs_subsample[targtype.upper()] =  sv1_cuts.isBGS(gflux=coadds['FLUX_G'] / coadds['MW_TRANSMISSION_G'],\
                                                        rflux=coadds['FLUX_R'] / coadds['MW_TRANSMISSION_R'],\
                                                        zflux=coadds['FLUX_Z'] / coadds['MW_TRANSMISSION_Z'],\
                                                        w1flux=coadds['FLUX_W1'] / coadds['MW_TRANSMISSION_W1'],\
                                                        w2flux=coadds['FLUX_W2'] / coadds['MW_TRANSMISSION_W2'],\
                                                        rfiberflux=coadds['FIBERFLUX_R']/ coadds['MW_TRANSMISSION_R'],\
                                                        gnobs=coadds['NOBS_G'],\
                                                        rnobs=coadds['NOBS_R'],\
                                                        znobs=coadds['NOBS_Z'],\
                                                        gfracmasked=coadds['FRACMASKED_G'],\
                                                        rfracmasked=coadds['FRACMASKED_R'],\
                                                        zfracmasked=coadds['FRACMASKED_Z'],\
                                                        gfracflux=coadds['FRACFLUX_G'],\
                                                        rfracflux=coadds['FRACFLUX_R'],\
                                                        zfracflux=coadds['FRACFLUX_Z'],\
                                                        gfracin=coadds['FRACIN_G'],\
                                                        rfracin=coadds['FRACIN_R'],\
                                                        zfracin=coadds['FRACIN_Z'],\
                                                        gfluxivar=coadds['FLUX_IVAR_G'],\
                                                        rfluxivar=coadds['FLUX_IVAR_R'],\
                                                        zfluxivar=coadds['FLUX_IVAR_Z'],\
                                                        maskbits=coadds['MASKBITS'],\
                                                        Grr=grr,\
                                                        w1snr=None,\
                                                        gaiagmag=coadds['GAIA_PHOT_G_MEAN_MAG'],\
                                                        objtype=coadds['MORPHTYPE'],\
                                                        primary=None,\
                                                        south=False,\
                                                        targtype=targtype)
    
    toprint['NIGHT'].append(night)
    toprint['TILE'].append(tile)
    toprint['NEXP'].append(key.ljust(8))
    
    for x in print_cols:
      toprint[x].append(results[tile][night][key][x])

    toprint['IS_GOOD'].append(np.count_nonzero(isgood))
    toprint['IS_BGS'].append(np.count_nonzero(IS_SV0))
    toprint['IS_GOOD_BGS'].append(np.count_nonzero(IS_GOOD_BGS))
    toprint['IS_ZGOOD_BGS'].append(np.count_nonzero(IS_ZGOOD_BGS))
    toprint['IS_ZGOOD_BGS_%'].append(100. * np.float(np.count_nonzero(IS_ZGOOD_BGS)) / np.float(np.count_nonzero(IS_GOOD_BGS)))
    toprint['IS_GOOD_MAIN'].append(np.count_nonzero(IS_GOOD_MAIN))
    toprint['IS_ZGOOD_MAIN'].append(np.count_nonzero(IS_ZGOOD_MAIN))  
    
    toprint['IS_GOOD_FAINT'].append(np.count_nonzero(bgs_subsample['FAINT']  & IS_GOOD_BGS))
    toprint['IS_ZGOOD_FAINT'].append(np.count_nonzero(bgs_subsample['FAINT']  & IS_ZGOOD_BGS))
    
    toprint['IS_GOOD_LOWQ'].append(np.count_nonzero(bgs_subsample['LOWQ']  & IS_GOOD_BGS))
    toprint['IS_ZGOOD_LOWQ'].append(np.count_nonzero(bgs_subsample['LOWQ']  & IS_ZGOOD_BGS))
    
    toprint['IS_GOOD_FIB'].append(np.count_nonzero(bgs_subsample['FIBMAG']  & IS_GOOD_BGS))
    toprint['IS_ZGOOD_FIB'].append(np.count_nonzero(bgs_subsample['FIBMAG']  & IS_ZGOOD_BGS))

    results[tile][night][key]['IS_BGS']       = IS_SV0
    results[tile][night][key]['IS_GOOD_BGS']  = IS_GOOD_BGS
    results[tile][night][key]['IS_ZGOOD_BGS'] = IS_ZGOOD_BGS 
    
##  
_ = pd.DataFrame()

for x in toprint.keys():
  try:
    _[x] = np.array(toprint[x])
    
  except:
    print('Failed for {}'.format(x))
_

/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:60: RuntimeWarning: invalid value encountered in true_divide
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:61: RuntimeWarning: invalid value encountered in true_divide
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:62: RuntimeWarning: invalid value encountered in true_divide
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:63: RuntimeWarning: invalid value encountered in true_divide
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:64: RuntimeWarning: invalid value encountered in true_divide
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packag

,NIGHT,TILE,NEXP,EXPTIME,MOON_FRAC,MOON_ALT,MOON_SEP,AIRMASS,SUN_ALT,SUN_SEP,...,IS_ZGOOD_BGS,IS_ZGOOD_BGS_%,IS_GOOD_MAIN,IS_ZGOOD_MAIN,IS_GOOD_FAINT,IS_ZGOOD_FAINT,IS_GOOD_LOWQ,IS_ZGOOD_LOWQ,IS_GOOD_FIB,IS_ZGOOD_FIB
0,20200225,70502,NEXP1-52115,900.0,0.066733,-54.733052,171.637603,1.217957,-65.621309,156.981431,...,1114,87.372549,746,719,258,212,10,2,65,42
1,20200225,70502,NEXP1-52114,900.0,0.066048,-51.968526,171.820477,1.250062,-66.729890,156.965316,...,1125,88.235294,746,721,258,214,10,1,65,48
2,20200225,70502,NEXP1-52113,900.0,0.065315,-48.586919,172.011711,1.299431,-66.796778,156.947803,...,1124,88.156863,746,720,258,218,10,1,65,49
3,20200225,70502,NEXP1-52116,900.0,0.067458,-57.144716,171.440369,1.196743,-63.508533,156.998249,...,1102,86.431373,746,713,258,207,10,3,65,46
4,20200225,70502,NEXP1-52112,900.0,0.064567,-44.783235,172.200971,1.369023,-65.647632,156.929632,...,1133,88.932496,745,720,258,218,10,1,65,49
5,20200225,70502,NEXP1-52090,900.0,0.062791,-34.718022,172.623066,1.649129,-58.985404,156.884725,...,1128,88.749017,742,718,258,221,10,2,65,48
6,20200225,70502,NEXP2-0,900.0,0.064419,-43.343274,172.221772,1.449596,-62.857647,156.925021,...,1156,90.595611,747,727,258,227,10,1,65,50
7,20200225,70502,NEXP2-1,900.0,0.066024,-51.659986,171.824657,1.258694,-66.209043,156.964617,...,1161,90.987461,747,724,258,228,10,1,65,52
8,20200225,70502,ALL-NEXP6,900.0,0.065485,-48.655745,171.955699,1.330391,-64.548258,156.951193,...,1181,92.554859,747,728,258,233,10,2,65,57
9,20200227,70502,NEXP1-52447,500.0,0.191811,-44.570970,149.349464,1.191672,-56.964642,159.040389,...,580,45.490196,746,429,258,80,10,0,65,26


##  Prospect

In [164]:
tile          = 70500
night         = '20200303'

keep          = results[tile][night]

In [166]:
_all                          = [x for x in keep.keys() if x[0] == 'A'][0]

# Keep ALL good such that targets are common across keys.
is_zgood_bgs                  = keep[_all]['IS_ZGOOD_BGS']

keys                          = keep.keys()
keys                          = [_all]

print(keys)

for key in keys:
  zbest                       = keep[key]['zbests']
  coadds                      = keep[key]['coadds']
    
  coadds_paths                = keep[key]['coadds_paths']

  zbest                       = Table(zbest)

  targetids                   = zbest['TARGETID']

  for petal in upetals:
    print('Generating key {} and petal {}.'.format(key, petal))
    
    specfile                  = coadds_paths[petal]
    
    spectra                   = desispec.io.read_spectra(specfile)
    spectra.fibermap["NIGHT"] = np.ones(len(spectra.fibermap))
    
    in_petal                  = is_zgood_bgs & (coadds['PETAL_LOC'] == petal)
    in_petal                  = targetids[in_petal]
    in_petal70                = in_petal[:70]
    
    sp_single_exp             = spectra.select(targets=in_petal70)

    plotframes.plotspectra(sp_single_exp,\
                           zcatalog=zbest,\
                           notebook=False,\
                           with_coaddcam=False,\
                           with_thumb_tab=False,\
                           title='{}_{}_'.format(tile, night) + key + '_' + str(petal),\
                           html_dir='/global/homes/m/mjwilson/www/MINISV')
    
_ = os.system('chmod --reference=/global/homes/m/mjwilson/www/plots/visibility-nofullmoon-26-0.pdf /global/homes/m/mjwilson/www/MINISV/*')

['ALL-NEXP12']
Generating key ALL-NEXP12 and petal 0.
Generating key ALL-NEXP12 and petal 3.
Generating key ALL-NEXP12 and petal 6.
Generating key ALL-NEXP12 and petal 7.
Generating key ALL-NEXP12 and petal 9.


0

In [155]:
_all                          = [x for x in keep.keys() if x[0] == 'A'][0]

for key in keep.keys():
 if keep[key] == None:
  continue

 else:
  assert   np.all(keep[_all]['zbests']['TARGETID'] == keep[key]['zbests']['TARGETID'])  

  zbest                       = keep[key]['zbests']
  coadds                      = keep[key]['coadds']
    
  fails                       = keep[key]['zbests']['ZWARN'] > 0  

  # Keep ALL good such that targets are common across keys.
  is_good_bgs                 = keep[key]['IS_GOOD_BGS']
  is_zgood_bgs                = keep[key]['IS_ZGOOD_BGS']

  isgood                      = keep[_all]['IS_ZGOOD_BGS'] & keep[key]['IS_ZGOOD_BGS']
  
  delta_v                     = 3.e5 * np.abs(keep[_all]['zbests']['Z'][isgood] - keep[key]['zbests']['Z'][isgood]) / (1. + keep[_all]['zbests']['Z'][isgood])    
  delta_v                     = delta_v > 1000.
  
  coadds_paths                = keep[key]['coadds_paths']

  zbest                       = Table(zbest)

  targetids                   = zbest['TARGETID'][isgood]
    
  for petal in upetals:    
    specfile                  = coadds_paths[petal]
    
    spectra                   = desispec.io.read_spectra(specfile)
    spectra.fibermap["NIGHT"] = np.ones(len(spectra.fibermap))
    
    in_petal                  = delta_v
    in_petal                  = targetids[in_petal]
    in_petal70                = in_petal[:] #70

    print('Generating cat. failures for night {} tile {} key {} and petal {}.  GOOD BGS: {}, ZGOOD_BGS {}'.format(night, tile, key, petal,\
                                                                                                                  np.count_nonzero(is_good_bgs & (coadds['PETAL_LOC'] == petal)),\
                                                                                                                  np.count_nonzero(is_zgood_bgs & (coadds['PETAL_LOC'] == petal))))
    
    try:
      sp_single_exp           = spectra.select(targets=in_petal70)
 
      plotframes.plotspectra(sp_single_exp, zcatalog=zbest, notebook=False, with_coaddcam=False, with_thumb_tab=False, title='{}_{}_'.format(tile, night) + key + '_' + str(petal) + '_CATFAILURES', html_dir='/global/homes/m/mjwilson/www/MINISV')
    
    except:
      print('\tNo valid targets were selected}.')
    
    #  Fix permissions.
    os.system('chmod --reference=/global/homes/m/mjwilson/www/plots/visibility-nofullmoon-26-0.pdf /global/homes/m/mjwilson/www/MINISV/*')
          
print('Done.')

Generating cat. failures for night 20200303 tile 70500 key NEXP1-52932 and petal 0.  GOOD BGS: 234, ZGOOD_BGS 205
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52932 and petal 3.  GOOD BGS: 205, ZGOOD_BGS 171
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52932 and petal 6.  GOOD BGS: 215, ZGOOD_BGS 188
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52932 and petal 7.  GOOD BGS: 215, ZGOOD_BGS 193
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52932 and petal 9.  GOOD BGS: 247, ZGOOD_BGS 203
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52933 and petal 0.  GOOD BGS: 234, ZGOOD_BGS 206
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52933 and petal 3.  GOOD BGS: 205, ZGOOD_BGS 176
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52933 and petal 6.  GOOD BGS: 215, ZGOOD_BGS 194
Generating cat. failures for night 20200303 tile 70500 key NEXP1-52933 and petal 7.  GOO

In [135]:
for key in keep.keys():
 if keep[key] == None:
  continue

 else:
  zbest                       = keep[key]['zbests']
  coadds                      = keep[key]['coadds']
    
  fails                       = keep[key]['zbests']['ZWARN'] > 0  

  # Keep ALL good such that targets are common across keys.
  is_good_bgs                 = keep[key]['IS_GOOD_BGS']
  is_zgood_bgs                = keep[key]['IS_ZGOOD_BGS']
        
  coadds_paths                = keep[key]['coadds_paths']

  zbest                       = Table(zbest)

  targetids                   = zbest['TARGETID']

  for petal in upetals:    
    specfile                  = coadds_paths[petal]
    
    spectra                   = desispec.io.read_spectra(specfile)
    spectra.fibermap["NIGHT"] = np.ones(len(spectra.fibermap))
    
    in_petal                  = is_good_bgs & fails & (coadds['PETAL_LOC'] == petal)
    in_petal                  = targetids[in_petal]
    in_petal70                = in_petal[:70]

    print('Generating failures for night {} tile {} key {} and petal {}.  GOOD BGS: {}, ZGOOD_BGS {}'.format(night, tile, key, petal,\
                                                                                                             np.count_nonzero(is_good_bgs & (coadds['PETAL_LOC'] == petal)),\
                                                                                                             np.count_nonzero(is_zgood_bgs & (coadds['PETAL_LOC'] == petal))))
    
    try:
      sp_single_exp           = spectra.select(targets=in_petal70)
 
      plotframes.plotspectra(sp_single_exp, zcatalog=zbest, notebook=False, with_coaddcam=False, with_thumb_tab=False, title='{}_{}_'.format(tile, night) + key + '_' + str(petal) + '_FAILURES', html_dir='/global/homes/m/mjwilson/www/MINISV')
    
    except:
      print('\tNo valid targets were selected}.')
    
    #  Fix permissions.
    os.system('chmod --reference=/global/homes/m/mjwilson/www/plots/visibility-nofullmoon-26-0.pdf /global/homes/m/mjwilson/www/MINISV/*')
          
print('Done.')

Generating failures for night 20200303 tile 70500 key NEXP1-52932 and petal 0.  GOOD BGS: 234, ZGOOD_BGS 205
Generating failures for night 20200303 tile 70500 key NEXP1-52932 and petal 3.  GOOD BGS: 205, ZGOOD_BGS 171
Generating failures for night 20200303 tile 70500 key NEXP1-52932 and petal 6.  GOOD BGS: 215, ZGOOD_BGS 188
Generating failures for night 20200303 tile 70500 key NEXP1-52932 and petal 7.  GOOD BGS: 215, ZGOOD_BGS 193


/global/homes/m/mjwilson/BGS/MINISV/prospect/py/prospect/plotframes.py:405: RuntimeWarning: All-NaN slice encountered
  ymin = min(ymin, np.nanmin(spectra.flux[band][0]))
/global/homes/m/mjwilson/BGS/MINISV/prospect/py/prospect/plotframes.py:406: RuntimeWarning: All-NaN slice encountered
  ymax = max(ymax, np.nanmax(spectra.flux[band][0]))


Generating failures for night 20200303 tile 70500 key NEXP1-52932 and petal 9.  GOOD BGS: 247, ZGOOD_BGS 203
Generating failures for night 20200303 tile 70500 key NEXP1-52933 and petal 0.  GOOD BGS: 234, ZGOOD_BGS 206
Generating failures for night 20200303 tile 70500 key NEXP1-52933 and petal 3.  GOOD BGS: 205, ZGOOD_BGS 176
Generating failures for night 20200303 tile 70500 key NEXP1-52933 and petal 6.  GOOD BGS: 215, ZGOOD_BGS 194
Generating failures for night 20200303 tile 70500 key NEXP1-52933 and petal 7.  GOOD BGS: 215, ZGOOD_BGS 199
Generating failures for night 20200303 tile 70500 key NEXP1-52933 and petal 9.  GOOD BGS: 247, ZGOOD_BGS 212
Generating failures for night 20200303 tile 70500 key NEXP1-52939 and petal 0.  GOOD BGS: 234, ZGOOD_BGS 207
Generating failures for night 20200303 tile 70500 key NEXP1-52939 and petal 3.  GOOD BGS: 202, ZGOOD_BGS 161
Generating failures for night 20200303 tile 70500 key NEXP1-52939 and petal 6.  GOOD BGS: 213, ZGOOD_BGS 189
Generating failures

##  Done.